<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con el cancionero de _Eminem_

A partir del ejercicio visto en clase, se puso a prueba la creación de embeddings de palabras basados en contexto. Su generación se realiza a través de la librería Gensim. 
Las pruebas consisten en ensayar para una misma palabra, cuáles del cancionero son cercanas (más relacionadas) y cuales son lejanas (menos relacionadas). Pueden ocurrir dos situaciones en función de los resultados:
- Palabras lejanas o cercanas por su significado.
- Si el contexto de la manera en qué están escritas las canciones influye sobre la condición dada por el significado que poseen las palabras. 

Hay que tener en cuenta que las letras de Eminem tienen una alta carga de giros idiomáticos y dialéctos propios del género del rap.



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Descarga del dataset.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-07-28 18:31:54--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-07-28 18:31:54--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-07-28 18:31:54--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.

In [6]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


### 1 - Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [10]:
# Demos un vistazo
sentence_tokens[:5]

[['look',
  'i',
  'was',
  'gonna',
  'go',
  'easy',
  'on',
  'you',
  'and',
  'not',
  'to',
  'hurt',
  'your',
  'feelings'],
 ['but', "i'm", 'only', 'going', 'to', 'get', 'this', 'one', 'chance'],
 ["something's",
  'wrong',
  'i',
  'can',
  'feel',
  'it',
  'six',
  'minutes',
  'slim',
  'shady',
  "you're",
  'on'],
 ['just',
  'a',
  'feeling',
  "i've",
  'got',
  'like',
  "something's",
  'about',
  'to',
  'happen',
  'but',
  'i',
  "don't",
  'know',
  'what'],
 ['if',
  'that',
  'means',
  'what',
  'i',
  'think',
  'it',
  'means',
  "we're",
  'in',
  'trouble',
  'big',
  'trouble']]

### 2 - Crear los vectores (word2vec)

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [13]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [14]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 6812


In [15]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 1347


### 3 - Entrenar el modelo generador

In [16]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 513593.84375
Loss after epoch 1: 369973.03125
Loss after epoch 2: 344217.125
Loss after epoch 3: 317115.0
Loss after epoch 4: 314096.5
Loss after epoch 5: 297876.25
Loss after epoch 6: 265934.0
Loss after epoch 7: 262286.5
Loss after epoch 8: 258583.5
Loss after epoch 9: 256254.5
Loss after epoch 10: 253678.5
Loss after epoch 11: 250982.5
Loss after epoch 12: 247370.5
Loss after epoch 13: 245402.25
Loss after epoch 14: 230377.5
Loss after epoch 15: 230240.0
Loss after epoch 16: 228948.0
Loss after epoch 17: 227543.5
Loss after epoch 18: 226511.5
Loss after epoch 19: 226759.0


(785273, 1296760)

### 4 - Pruebas
Se ensayan palabras que son sustantivos. Y por cada palabra se observa con cuales posee mayor o menor relación.

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["car"], topn=10)

[('cell', 0.8657675385475159),
 ('bridge', 0.8479256629943848),
 ('stack', 0.8301877975463867),
 ('aiming', 0.8283246755599976),
 ('state', 0.826807975769043),
 ('post', 0.8245569467544556),
 ("jumpin'", 0.8240019679069519),
 ('wife', 0.8196628093719482),
 ('bedroom', 0.8167269229888916),
 ('headlights', 0.8126990795135498)]

In [34]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["home"], topn=10)

[('do', -0.011182699352502823),
 ('know', -0.11043538898229599),
 ('like', -0.13565483689308167),
 ('was', -0.14658835530281067),
 ("ain't", -0.15297147631645203),
 ("actin'", -0.1547769010066986),
 ('been', -0.15578427910804749),
 ('be', -0.1593363881111145),
 ('fuck', -0.16521385312080383),
 ("don't", -0.16593526303768158)]

**_Sustantivos_**
- car: de las palabras más relaciodas aparecen 'bridge', 'headlights', 'post'. Son palabras que podría pensarse que podría utilizarse con 'car'. En cambio 'wife' o 'aiming' dependen de las letras de las canciones. Entre las menos relacionadas llama la atención que aparezcan algunos adverbios como 'ever' o 'sometimes'. 'want' también aparece entre las menos relacionadas.
- home: entre las palabras más relacionadas por su uso en significado se encuentran 'lives', 'parents', 'grow'. Las que dependen de las letras de las canciones son 'paid', 'shows'. En cuanto a las palabras menos relacionadas es dificil encontrar alguna explicación desde el significado. Aparecen 'fuck', 'like', 'know'. Llama la atención que aparezcan el verbo to be y el auxiliar to do de diferentes formas: 'was', 'do', 'was', 'don't'.

Aparecen otras palabras en forma de giro idiomático, algunas veces recortadas, como es el caso de 'jumpin'. Por este motivo no se probaron verbos, ya que requiere conocer mejor como es el dialecto empleado en inglés. Se destaca que entre las palabras menos relacionadas aparezcan adverbios y pronombres o formas lingüisticas más desconocidas. 

Por último se visualiza la agrupación de vectores.


### 5 - Visualizar agrupación de vectores

In [22]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [23]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
